In [2]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

load_dotenv()
llm=ChatOpenAI(model="gpt-4o-mini")

llm.invoke([HumanMessage("잘지냈어?")])

AIMessage(content='네, 잘 지냈습니다! 당신은 어떻게 지내고 계신가요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 12, 'total_tokens': 30, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_c4585b5b9c', 'id': 'chatcmpl-CxTfC7rzErk30qdKhak8x8ULnhE95', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019bb661-abd1-77e1-9888-1ef62708c5f2-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 12, 'output_tokens': 18, 'total_tokens': 30, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [5]:
from langchain_core.tools import tool
from datetime import datetime
import pytz

@tool(description="타임과 지역명을 받아 현재 시각을 문자열로 반환하는 도구")
#@tool 데코레이터를 사용해 함수를 도구로 등록함
def get_current_time(timezone: str, location: str) -> str:
    tz=pytz.timezone(timezone)
    now=datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S")
    location_and_local_time=f'{timezone} ({location}) 현재 시각 {now}'
    #타임존, 지역명, 현재 시각을 문자열로 반환함
    print(location_and_local_time)
    return location_and_local_time


In [6]:
#도구를 tools 리스트에 추가, tool_dict에도 추가
tools=[get_current_time,]
tool_dict={"get_current_time": get_current_time,}

#도구를 모델에 바인딩: 모델에 도구를 바인딩하면, 도구를 사용하여 답변을 생성할 수 있음
llm_with_tools=llm.bind_tools(tools)

In [7]:
from langchain_core.messages import SystemMessage

#사용자의 질문과 도구를 사용해 언어 모델 답변 생성
messages=[
    SystemMessage("너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다."),
    HumanMessage("부산은 지금 몇 시야?"),
]

#llm_with_tools를 사용해 사용자의 질문에 언어 모델 답변 생성
response = llm_with_tools.invoke(messages)
messages.append(response)

#생성된 언어 모델 답변 출력
print(messages)

#GPT가 함수 한 번만 사용하면 된다고 판단했으므로, response.tool_calls를 확인하면 1개만 들어있음음

[SystemMessage(content='너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='부산은 지금 몇 시야?', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 84, 'total_tokens': 107, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_c4585b5b9c', 'id': 'chatcmpl-CxTj3N8SKBsQuSukTH0XmhakGV1YC', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019bb665-54bc-7c61-9fe1-b0b98686764a-0', tool_calls=[{'name': 'get_current_time', 'args': {'timezone': 'Asia/Seoul', 'location': '부산'}, 'id': 'call_8qItQVKFtaeI2sbA1QMygbbN', 'type': 'tool_call'}],

In [8]:
for tool_call in response.tool_calls:
    selected_tool = tool_dict[tool_call["name"]]
    #tool_dict를 사용해 도구 함수 선택
    print(tool_call["args"])
    #도구 호출시 전달된 인자를 출력함
    tool_msg = selected_tool.invoke(tool_call)
    #도구 함수를 호출해 결과를 반환함
    messages.append(tool_msg)

messages

{'timezone': 'Asia/Seoul', 'location': '부산'}
Asia/Seoul (부산) 현재 시각 2026-01-13 17:09:52


[SystemMessage(content='너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='부산은 지금 몇 시야?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 84, 'total_tokens': 107, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_c4585b5b9c', 'id': 'chatcmpl-CxTj3N8SKBsQuSukTH0XmhakGV1YC', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019bb665-54bc-7c61-9fe1-b0b98686764a-0', tool_calls=[{'name': 'get_current_time', 'args': {'timezone': 'Asia/Seoul', 'location': '부산'}, 'id': 'call_8qItQVKFtaeI2sbA1QMygbbN', 'type': 'tool_call'}

In [9]:
llm_with_tools.invoke(messages)

AIMessage(content='부산의 현재 시각은 2026년 1월 13일 오후 5시 9분입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 140, 'total_tokens': 167, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_29330a9688', 'id': 'chatcmpl-CxTltatGFpuXZN46tmqXVgBDtiIYd', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019bb668-0051-71a2-949e-5de6946b713f-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 140, 'output_tokens': 27, 'total_tokens': 167, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})